In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\abhij\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [4]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [5]:
num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [6]:
def scale(image , label):
    image = tf.cast(image , tf.float32)
    image = image / 255
    return image , label

scaled_train_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [7]:
# let's shuffle the data

buffer_size = 1000

shuffled_train_validation_data = scaled_train_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_validation_data.take(num_validation_samples)

train_data = shuffled_train_validation_data.skip(num_validation_samples)

In [8]:
# let's batch the data which will help while implementing gradient decsnet with faster learning rate

batch_size = 100

train_data = train_data.batch(batch_size)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

In [9]:
# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

In [10]:
# let's create the model

input_size = 784
hidden_layer_size = 200

output_size = 10

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

epoch_size = 15

model.fit(train_data, epochs = epoch_size, validation_data =(validation_inputs, validation_targets), verbose = 2)

Epoch 1/15
540/540 - 7s - loss: 0.2647 - accuracy: 0.9201 - val_loss: 0.1557 - val_accuracy: 0.9522
Epoch 2/15
540/540 - 2s - loss: 0.1048 - accuracy: 0.9684 - val_loss: 0.1187 - val_accuracy: 0.9637
Epoch 3/15
540/540 - 2s - loss: 0.0738 - accuracy: 0.9781 - val_loss: 0.1291 - val_accuracy: 0.9648
Epoch 4/15
540/540 - 2s - loss: 0.0591 - accuracy: 0.9817 - val_loss: 0.1268 - val_accuracy: 0.9628
Epoch 5/15
540/540 - 2s - loss: 0.0507 - accuracy: 0.9841 - val_loss: 0.1624 - val_accuracy: 0.9575
Epoch 6/15
540/540 - 2s - loss: 0.0400 - accuracy: 0.9874 - val_loss: 0.0938 - val_accuracy: 0.9758
Epoch 7/15
540/540 - 2s - loss: 0.0364 - accuracy: 0.9886 - val_loss: 0.1078 - val_accuracy: 0.9737
Epoch 8/15
540/540 - 2s - loss: 0.0333 - accuracy: 0.9895 - val_loss: 0.1237 - val_accuracy: 0.9718
Epoch 9/15
540/540 - 2s - loss: 0.0263 - accuracy: 0.9919 - val_loss: 0.1240 - val_accuracy: 0.9735
Epoch 10/15
540/540 - 2s - loss: 0.0284 - accuracy: 0.9907 - val_loss: 0.1097 - val_accuracy: 0.9752

In [11]:
# test_model

test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 460ms/step - loss: 0.1238 - accuracy: 0.9763


In [12]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.12. Test accuracy: 97.63%
